## Daten aus mysql in mongodb importieren

In [ ]:
# import pymongo
# import mysql.connector
# import schedule
# import time

# def load_data():
#     # Verbindung zur MongoDB herstellen
#     mongo_client = pymongo.MongoClient("mongodb://localhost:27017/")
#     mongo_db = mongo_client['deine_mongo_db']
#     mongo_collection = mongo_db['deine_mongo_collection']

#     # Verbindung zu MySQL herstellen
#     mysql_connection = mysql.connector.connect(
#         host="localhost",
#         user="dein_mysql_benutzer",
#         password="dein_mysql_passwort",
#         database="deine_mysql_db"
#     )
#     mysql_cursor = mysql_connection.cursor()

#     # MongoDB-Daten abrufen
#     mongo_data = mongo_collection.find()

#     # MySQL-Tabelle erstellen (falls noch nicht vorhanden)
#     mysql_cursor.execute("CREATE TABLE IF NOT EXISTS deine_mysql_tabelle (col1 VARCHAR(255), col2 INT)")

#     # MongoDB-Daten in MySQL einfügen
#     for document in mongo_data:
#         col1_value = document.get("field1", "")
#         col2_value = document.get("field2", 0)
#         mysql_cursor.execute("INSERT INTO deine_mysql_tabelle (col1, col2) VALUES (%s, %s)", (col1_value, col2_value))

#     # Änderungen in der MySQL-Datenbank speichern
#     mysql_connection.commit()

#     # Verbindungen schließen
#     mysql_cursor.close()
#     mysql_connection.close()
#     mongo_client.close()

# # Zeitplan für alle 30 Minuten einrichten
# schedule.every(30).minutes.do(load_data)

# while True:
#     schedule.run_pending()
#     time.sleep(1)


## Every business has a unique business_id and they could just see the reviews of their business

In [2]:
# connect the business_name and business address with the business_id to fetch the review from the dusiness_id
import pymongo
from bson.objectid import ObjectId

# Establish connection to MongoDB
client = pymongo.MongoClient("mongodb://localhost:27017/")
# Access the database
db = client['yelp']

# Access the collections
review_collection = db['review']
business_collection = db['business']
admin_collection = db['admin']


def authenticate_user(admin_username, admin_password, business_name, business_address):
    # Find the admin based on username and password
    admin = admin_collection.find_one({"admin_username": admin_username, "admin_password": admin_password})
    if not admin:
        return "Authentication failed. Invalid username or password."

    # Find or create a unique business_id based on business_name and business_address
    business = business_collection.find_one({"name": business_name, "address": business_address})
    if not business:
        business_id = str(ObjectId())
        business_collection.insert_one({
            "business_id": business_id,
            "name": business_name,
            "address": business_address
        })
    else:
        business_id = business['business_id']

    # Check if the authenticated admin is associated with the specified business
    if admin['business_id'] == business_id:
        # Retrieve reviews for the specified business_id
        reviews = review_collection.find({"business_id": business_id})
        return list(reviews)
    else:
        return "Authentication failed. Admin does not have access to the specified business."

# Beispielaufruf:
admin_username = input("Admin Username: ")
admin_password = input("Admin Password: ")
business_name = input("Business Name: ")
business_address = input("Business Address: ")

result = authenticate_user(admin_username, admin_password, business_name, business_address)
print(result)

[{'_id': ObjectId('656cc6c15d1141c25dc99f63'), 'review_id': '419RpsM5N8Mr-QkpuPxi5Q', 'user_id': 'gIzS3s4yZm6WiXM3TcQMPQ', 'business_id': 'CF33F8-E6oudUQ46HnavjQ', 'stars': 4, 'useful': 0, 'funny': 0, 'cool': 0, 'text': 'I have always liked Sonic as it is good fresh food as far as fast food goes. Later in the evenings this location is a little slow.', 'date': '2012-12-18 08:45:44'}, {'_id': ObjectId('656cc6c45d1141c25dca9794'), 'review_id': '2mD-VlkQqErngk7xwjht1g', 'user_id': 'zF10BKjK4Lz9U_8Yqw28ww', 'business_id': 'CF33F8-E6oudUQ46HnavjQ', 'stars': 4, 'useful': 0, 'funny': 0, 'cool': 0, 'text': "How could I not like Sonic?  They have a Peanut Butter Shake.  I've had a hard time finding those, so I always enjoy resisting the urge to actually get one.  My waistline clearly shows how often I am successful at this.\n\nYou really can't go wrong with Sonic!", 'date': '2013-07-08 00:59:08'}, {'_id': ObjectId('656cc6c45d1141c25dcaa798'), 'review_id': 'g4tFQ557ubgN_So9uFiSaQ', 'user_id': 'J9

## Query abfragung mit mysql

In [4]:
import mysql.connector

# Verbindung zur MySQL-Datenbank herstellen
db = mysql.connector.connect(
    host="localhost",
    user="dein_benutzername",
    password="dein_passwort",
    database="deine_datenbank"
)

# Beispielabfrage: Alle Bewertungen für ein bestimmtes Unternehmen in einem bestimmten Zeitraum
business_name = "Abby Rappoport, LAC, CMQ"
start_date = "2023-01-01"
end_date = "2023-12-31"

cursor = db.cursor()
query = f"SELECT * FROM review WHERE business_name = '{business_name}' AND date BETWEEN '{start_date}' AND '{end_date}'"
cursor.execute(query)

# Ergebnisse anzeigen
for review in cursor.fetchall():
    print(review)


C:\Users\Tristan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\mysql\connector\abstracts.py:130: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if group is 'connector_python':
C:\Users\Tristan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\mysql\connector\abstracts.py:130: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if group is 'connector_python':


InterfaceError: 2003: Can't connect to MySQL server on 'localhost:3306' (10061 No connection could be made because the target machine actively refused it)

## Query abfragung mit MongoDB

In [3]:
import pymongo
from datetime import datetime

# Establish connection to MongoDB
client = pymongo.MongoClient("mongodb://localhost:27017/")
# Access the database
db = client['yelp']

# Access the collection
review_collection = db['review']

# Query parameters
business_id = "CF33F8-E6oudUQ46HnavjQ"
start_date = datetime(2023, 1, 1, 0, 0, 0)
end_date = datetime(2023, 12, 31, 23, 59, 59)

# Actual query with specified filters
reviews = review_collection.find({
    "business_id": business_id,
    "date": {"$gte": start_date, "$lte": end_date}
})

# Print query results
print("Query results (with filters):")
print(list(reviews))
blah

Query results (with filters):
[]
